In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'1.4.2'

In [3]:
try:
    df = pd.read_xml("p:/PrintOut_StatReportForExport.xml", xpath="//IHComTable")
except:
    df = pd.read_xml("PrintOut_StatReportForExport.xml", xpath="//IHComTable")

    
now_string = pd.Timestamp.now().strftime('%Y%m%d_%H%M')

try:
    df.to_excel(f"F:/Printer/ALL_{now_string}.xlsx")
except OSError:
    df.to_excel(f"ALL_{now_string}.xlsx")

In [4]:
# qc only
qc_df = df[df['SampleBarcode'].str.contains("QC")]
qc_df = qc_df.copy()
qc_df['BGText'] = qc_df['BGText'].fillna("")
qc_df['BGTextABScrDAT'] = qc_df['BGTextABScrDAT'].fillna("")
# remove not interpretable
qc_df = qc_df[~(qc_df['BGText'].str.contains("not"))]
qc_df = qc_df[~(qc_df['BGTextABScrDAT'].str.contains("not"))]

In [5]:
group = qc_df.groupby(['SampleBarcode','TestDate'])


In [6]:
sample_df_list=[]

for key in group.groups.keys():
    sample_df = group.get_group(key).set_index("WellName")
    sample_result_series = sample_df.FinalResultText
    
    sample_df['ResultText'] = sample_df['BGText']+sample_df['BGTextABScrDAT']
    sample_info_series = sample_df.iloc[0][['TestDate','SampleBarcode','VerifiedByUser', 'ResultText']]

    sample_df_list.append(pd.DataFrame(pd.concat([sample_info_series, sample_result_series])).T)
    

In [7]:
result_df = pd.concat(sample_df_list)
result_df=result_df[~result_df['VerifiedByUser'].isna()]
result_df.fillna("", inplace=True)
result_df = result_df.replace("DP", np.nan).dropna()
result_df.set_index("TestDate", inplace=True)

result_df.sort_index(inplace=True)
result_df.reset_index(inplace=True)

In [8]:


try:
    result_df.to_excel(f"F:/Printer/QC_{now_string}.xlsx")
except OSError:
    result_df.to_excel(f"QC_{now_string}.xlsx")
    

In [9]:
print(result_df)

                          TestDate     SampleBarcode          VerifiedByUser  \
0    2022-03-01T00:43:44.599+08:00  08710671220307QC  automatically accepted   
1    2022-03-01T00:43:55.609+08:00  08720671220307QC  automatically accepted   
2    2022-03-01T00:44:23.131+08:00  08720671220307QC  automatically accepted   
3    2022-03-01T00:44:23.185+08:00  08770671220307QC  automatically accepted   
4    2022-03-01T00:44:30.952+08:00  08710671220307QC  automatically accepted   
..                             ...               ...                     ...   
338  2022-03-31T22:30:39.736+08:00  08710691220404QC  automatically accepted   
339  2022-03-31T22:30:51.395+08:00  08720691220404QC  automatically accepted   
340  2022-03-31T22:31:03.767+08:00  08720691220404QC  automatically accepted   
341  2022-03-31T22:31:12.059+08:00  08710691220404QC  automatically accepted   
342   2022-03-31T22:31:12.21+08:00  08720691220404QC  automatically accepted   

          ResultText Anti-A Anti-B Anti

In [10]:
dc_screening_df = result_df[result_df['IgG']!=""][["TestDate","SampleBarcode","VerifiedByUser","ResultText","Ctl", "IgG", "C3d"]]
dc_screening_df

,TestDate,SampleBarcode,VerifiedByUser,ResultText,Ctl,IgG,C3d
32,2022-03-03T03:36:55.08+08:00,08770671220307QC,automatically accepted,DAT positive,-,+++,-
33,2022-03-03T03:36:55.239+08:00,08780671220307QC,automatically accepted,DAT positive,-,-,+++
115,2022-03-10T07:32:02.862+08:00,08770691220404QC,automatically accepted,DAT positive,-,+++,-
116,2022-03-10T07:32:03.185+08:00,08780691220404QC,automatically accepted,DAT positive,-,-,+++
199,2022-03-16T18:26:02.670928+08:00,08770691220404QC,automatically accepted,DAT positive,-,++,-
200,2022-03-16T18:26:02.670928+08:00,08780691220404QC,automatically accepted,DAT positive,-,-,++


In [11]:
result_html = result_df.to_html(index=False)
dc_html = dc_screening_df.to_html(index=False)

In [12]:
start_date = result_df['TestDate'].apply(pd.to_datetime).min().strftime('%Y/%m/%d')
end_date  = result_df['TestDate'].apply(pd.to_datetime).max().strftime('%Y/%m/%d')

title = f"{start_date}-{end_date}"

In [18]:
html_string = '''

<!DOCTYPE html>
<html>
<head>
<title>{time_range}</title>

{style}


</head>
<body>

<h1>成大醫院 病理部 血庫組 {title} 品管報表</h1>

<p>
<table width="100%">
	<thead>
		<tr style="text-align:right; font-size:12px; font-family:DFKai-sb;">
			<td>組長:_______________________</td>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td> 
                
                {result}

            </td>
		</tr>
	</tbody>
</table>
</p>




</body>
</html>

'''

# print footer
'''
	@media print {
	  footer {
		position: fixed;
		bottom: -15px;
		right: 30px;
		width:100%
		border: 1px solid red;
		font-size: 15px;
		font-family: DFKai-sb;
	  }
       .content-block, p {
		page-break-inside: avoid;
	  }
	}
    
'''
        
style_string='''
<style>
    @media print {
      h1 {page-break-before: always;}
    }
    
    @media screen{
        body {
            width:90%;
            margin: auto auto;
        }
    }

    
	table, th, td{
		border: 0;
	}
    
    h1 {
		text-align:center;
		font-family:DFKai-sb;
		font-size: 20px;
	}
	
	.dataframe {
		width: 100%;
		font-family: calibri;
		font-size: 10px;
		border-bottom: 2px solid black;
		border-top: 2px solid black;
	}
	
	.dataframe th{
		border-bottom: 1px solid black;
		text-align: center;
	}
	
	.dataframe td:nth-child(-n+4){
		text-align: left;
	}
	
	.dataframe td{
		text-align: center;
	}
	
	.dataframe tr:nth-child(even){
		background-color: #f2f2f2;
	}
	
</style>
'''

In [19]:
with open("ih500.html", "w") as f:
     f.write(html_string.format(result=result_html,
                               style=style_string,
                               time_range=title,
                               title="IH-500"))
        
with open("dc_screening.html", "w") as f:
     f.write(html_string.format(result=dc_html,
                               style=style_string,
                               time_range=title,
                               title="DC Screening-II"))
    


In [15]:
def check_qc(_series):
    barcode = _series['SampleBarcode']
    anti_A = _series['Anti-A']
    result_text = _series['ResultText']
    dat = _series['DAT poly']
    mono = _series['C3d']
    screen = _series['I']
    
    # qc1
    if barcode.startswith("0871"):
            if anti_A!="":
                return result_text == "A Rh D negative"
            
            if screen !="":
                return result_text == "ABS positive"
    
    # qc2
    if barcode.startswith("0872"):
            if anti_A!="":
                return result_text == "B Rh D positive"
            
            if screen !="":
                return result_text == "ABS positive"
            
            if dat !="":
                return result_text == "DAT negative"
    
    # qc7
    if barcode.startswith("0877"):           
            if dat !="":
                return result_text == "DAT positive"
            
            if mono !="":
                return result_text == "DAT positive"
            
    # qc8
    if barcode.startswith("0878"):           
            if dat !="":
                return result_text == "DAT positive"
            
            if mono !="":
                return result_text == "DAT positive"
            
    
    print(_series)

In [16]:
result_df['QC_check'] = result_df.apply(check_qc, axis=1)

In [17]:
print(result_df['QC_check'].unique())

[ True]
